In [170]:
## load required libraries 
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from pdpbox import pdp, get_dataset, info_plots
from eli5.sklearn import PermutationImportance
from sklearn.preprocessing import LabelEncoder
from collections import Counter
import pandas as pd 
import numpy as np 
import eli5
import re



In [198]:
#Load the dataset
df = pd.read_csv("../input/ks-projects-201801.csv",  parse_dates = ["launched", "deadline"])
df_copy = df.copy()  #keep a copy of the data frame that remains intact
print ("Number of Campaigns: ", projects.shape[0], "\nNumber of Columns: ", projects.shape[1])
df.head()



Number of Campaigns:  378661 
Number of Columns:  15


,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09,1000.0,2015-08-11 12:12:28,0.0,failed,0,GB,0.0,0.0,1533.95
1,1000003930,Greeting From Earth: ZGAC Arts Capsule For ET,Narrative Film,Film & Video,USD,2017-11-01,30000.0,2017-09-02 04:43:57,2421.0,failed,15,US,100.0,2421.0,30000.00
2,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26,45000.0,2013-01-12 00:20:50,220.0,failed,3,US,220.0,220.0,45000.00
3,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16,5000.0,2012-03-17 03:24:11,1.0,failed,1,US,1.0,1.0,5000.00
4,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29,19500.0,2015-07-04 08:35:03,1283.0,canceled,14,US,1283.0,1283.0,19500.00


As of every machine learning problem, we need to explore the data and look out for problematic cases. These steps can be listed below:
* Outliers
* Missing values
* Unwanted information/labels
* Encode categorical values to numerical
* Extract Features

Let's see if there are any missing values. We will deal with outliers by considering features relevant to mean and median of each category. 

In [199]:
df.isnull().sum().sort_values(ascending = False)

usd pledged         3797
name                   4
usd_goal_real          0
usd_pledged_real       0
country                0
backers                0
state                  0
pledged                0
launched               0
goal                   0
deadline               0
currency               0
main_category          0
category               0
ID                     0
dtype: int64

So as it shows, usd_pledged has around 1% of it's values missing and there are 4 campaigns with missing names. Let's take a further look.

In [200]:
display(df[df['usd pledged'].isnull()].shape)
df[df['usd pledged'].isnull()].head()

(3797, 15)

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real,usd_goal_real
169,1000694855,STREETFIGHTERZ WHEELIE MURICA,Film & Video,Film & Video,USD,2014-09-20,6500.0,2014-08-06 21:28:36,555.00,undefined,0,"N,0""",NaN,555.00,6500.00
328,100149523,Duncan Woods - Chameleon EP,Music,Music,AUD,2015-08-25,4500.0,2015-08-04 12:05:17,4767.00,undefined,0,"N,0""",NaN,3402.08,3211.53
632,1003023003,The Making of Ashley Kelley's Debut Album,Music,Music,USD,2015-04-09,3500.0,2015-03-10 20:06:13,3576.00,undefined,0,"N,0""",NaN,3576.00,3500.00
647,1003130892,Butter Side Down Debut Album,Music,Music,USD,2015-11-26,6000.0,2015-11-02 22:09:19,7007.80,undefined,0,"N,0""",NaN,7007.80,6000.00
749,1003629045,Chase Goehring debut EP,Music,Music,USD,2016-03-21,3000.0,2016-02-23 03:09:49,3660.38,undefined,0,"N,0""",NaN,3660.38,3000.00


It seems reasonable to keep these 3797 rows and substitute the NAN values with the corresponding usd_pledged_real and drop those 4 rows with no name. 
Also a glance at the country column shows that some country entries seem to be incorrect with the '*N,0"*' entry. We should also consider taking care of that. 
Since most of the campaigns are from the states (78.85%) we will use a indicator of being from the states or not. Hence these values will turn to 0 (not from the states, assuming that '*N,0"*' does not indicate that).

In [201]:
df['usd pledged'].fillna(df['usd_pledged_real'], inplace=True)
df.dropna(inplace=True)

In [202]:
#Let's make sure the null values have been taken care of:
df.isnull().sum().sort_values(ascending = False)

usd_goal_real       0
usd_pledged_real    0
usd pledged         0
country             0
backers             0
state               0
pledged             0
launched            0
goal                0
deadline            0
currency            0
main_category       0
category            0
name                0
ID                  0
dtype: int64

Great! So now that we have dealt with the missing values, let's delve into the feature extraction step. Below is a list of useful features that we will be using:
*Features focusing on the name of the campaign can be:*
* Number of words in the name
* Number of syllables
* Presence of particular punctuation (e.g. ? !)
* Number of characters
* Ratio between vowels and alphanumeric length of the name
* Whether the name is uppercase or not (Maybe names in uppercase can attract more attention to potential backers)

These features are useful because we are interested to know whether a specific range of words, characters or any sort of pattern is effective in the campaign's success.
Other important features could be those relevant to the time of launch and its durations. There may be a correlation between the time and day/month a campaign is launched.
Some features in this aspect could be:
* Launched hour
* Launched day
* Launched week
* Launched month
* Whether it was a weekend or not
* Duration of the campaign 

Another group of features which is obviously effective is based on the goal and pledged amount. We can extract new features by considering the mean goal in each category and main category to see if a specific campaign is an outlier compared to other campaignes of its field, etc. Below is a list of such features:
* Mean of the category's goal
* Median of the category's goal
* Count of the categoty (to see whether the campaign is relatively popular or not)
* Mean of the main category's goal
* Median of the main category's goal
* Count of the main category
* Difference between a campaign's goal and mean of its category
* Pledged amount as a percentage of the goal
* Pledged amount per backer 
* Average Pledged amount per category 
* Difference between pledged amount per backer for a campaign and the average received in that category

Another interesting feature could be the country where the campaign is launched.

The following blocks computes the features listed above.

In [203]:
 #Let's only keep the successful and failed labels and turn it into a binary classification:
df.query("state in ['failed', 'successful']", inplace=True)

In [204]:
def syllable_count(word):
    """Count the syllables in a word"""
    word = word.lower()
    vowels = "aeiouy"
    count = 0
    if word[0] in vowels:
        count += 1
    for index in range(1, len(word)):
        if word[index] in vowels and word[index - 1] not in vowels:
            count += 1
    if word.endswith("e"):
        count -= 1
    if count == 0:
        count += 1
    return count

In [205]:
def countVowels2LettersRatio(word):
    '''Count ratio between vowels and letters'''
    word = str(word)
    count = 1  
    vowels = 0
    for i in word:
        if i.isalpha():
            count = count + 1
            if i in 'aeiou':
                vowels = vowels + 1
    return ((vowels * 1.0) / count)



In [206]:
## feature extraction for the campaign name

df["num_syllable"] = df["name"].apply(lambda x: syllable_count(x))
df["num_words"] = df["name"].apply(lambda x: len(x.split()))
df["num_chars"] = df["name"].apply(lambda x: len(x.replace(" ","")))
df['has_exclaimation_mark'] = int(len(re.findall(r'\!', str(df.name.str[:]))) != 0)  #presence of ! in the name
df['has_question_mark'] = int(len(re.findall(r'\?', str(df.name.str[:]))) != 0)  #presence of ? in the name
df['name_is_upper'] = df.name.str.isupper().astype(float)# if name is uppercase
df['name_vowel_ratio'] = df.name.apply(countVowels2LettersRatio)# for each name calculate vowels ratio




## feature extraction for the campaign length and timing
df["launched_month"] = df["launched"].dt.month
df["launched_week"] = df["launched"].dt.week
df["launched_day"] = df["launched"].dt.weekday
df["is_weekend"] = df["launched_day"].apply(lambda x: 1 if x > 4 else 0)
df["duration_in_days"] = df["deadline"] - df["launched"]
df["duration_in_days"] = df["duration_in_days"].apply(lambda x: int(str(x).split()[0]))

## label encoding the categorical features. This is necessary for the features related to goal and pledged amounts
#df = pd.concat([df, pd.get_dummies(df["main_category"])], axis = 1)
label_encoder = LabelEncoder()
for cat in ["category", "main_category"]:
    df[cat] = label_encoder.fit_transform(df[cat])
   
## feature extraction for goal and pledged values
df.head()
c1 = df.groupby("main_category").agg({"goal" : "mean", "category" : "sum"})
c2 = df.groupby("category").agg({"goal" : "mean", "main_category" : "sum"})
c1 = c1.reset_index().rename(columns={"goal" : "main_category_goal_mean", "category" : "main_category_count"})
c2 = c2.reset_index().rename(columns={"goal" : "category_goal_mean", "main_category" : "category_count"})
df = df.merge(c2, on = "category")
df = df.merge(c1, on = "main_category") 
df["diff_mean_category_goal"] = df["category_goal_mean"] - df["goal"]
df["diff_mean_category_goal"] = df["main_category_goal_mean"] - df["goal"]
df['goal_log'] = np.log1p(df.goal) # normalizing goal by applying log to prevent skewness
df.loc[:,'goal_reached'] = df['pledged'] / df['goal'] # Pledged amount as a percentage of goal.
df.loc[df['backers'] == 0, 'backers'] = 1 # In backers column, impute 0 with 1 to prevent undefined division.
df.loc[:,'pledge_per_backer'] = df['pledged'] / df['backers'] # Pledged amount per backer.

## create indicator variable for `country` variable
df['country_is_us'] = (df.country == 'US').astype(int)


In [207]:
df["state"] = df["state"].apply(lambda x: 1 if x=="successful" else 0)


In [208]:

df = df.drop(["launched", "deadline", "name", "currency", "country"], axis = 1) #drop the unnecassary columns

In [214]:
#Define the label and features and scale the data
label = df.state
features = [c for c in df.columns if c not in ["state"]]
#Let's normalise the data entries
df_scaled = pd.DataFrame(sklearn.preprocessing.normalize(df[features]))
#df_scaled.columns=list(df_scaled[features])

In [216]:
df_scaled.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
0,0.999997,1.079994e-07,1.199994e-08,9.999947e-07,0.000000e+00,9.999947e-10,0.000000e+00,0.000000e+00,1.533942e-06,9.999947e-09,5.999968e-09,2.599986e-08,9.999947e-10,9.999947e-10,0.0,3.461520e-10,7.999958e-09,3.299983e-08,9.999947e-10,0.0,5.799969e-08,0.000005,0.000015,0.000019,0.002438,0.000018,6.908718e-09,0.000000e+00,0.000000e+00,0.000000e+00
1,0.999997,1.078888e-07,1.198764e-08,3.995880e-06,1.997940e-08,9.989699e-10,3.062842e-08,3.026879e-08,6.054727e-06,1.298661e-08,7.991759e-09,4.295571e-08,9.989699e-10,9.989699e-10,0.0,3.567750e-10,5.993820e-09,2.597322e-08,3.995880e-09,0.0,2.897013e-08,0.000005,0.000015,0.000019,0.002435,0.000015,8.285756e-09,4.994850e-12,1.997940e-08,0.000000e+00
2,0.999997,1.078414e-07,1.198237e-08,1.997062e-06,3.245226e-07,5.991187e-09,3.245226e-07,3.245226e-07,1.997062e-06,3.994124e-09,2.995593e-09,1.397944e-08,9.985311e-10,9.985311e-10,0.0,2.662750e-10,2.995593e-09,9.985311e-09,3.994124e-09,0.0,2.895740e-08,0.000005,0.000015,0.000019,0.002434,0.000017,7.590237e-09,1.622613e-10,5.408710e-08,9.985311e-10
3,0.999997,1.077850e-07,1.197611e-08,9.980095e-06,1.007990e-07,1.996019e-09,1.007990e-07,1.007990e-07,9.980095e-06,1.497014e-08,8.982086e-09,4.391242e-08,9.980095e-10,9.980095e-10,0.0,3.855946e-10,4.990048e-09,1.796417e-08,2.994029e-09,0.0,2.894228e-08,0.000005,0.000015,0.000019,0.002433,0.000009,9.192107e-09,1.007990e-11,5.039948e-08,9.980095e-10
4,0.999997,1.077283e-07,1.196981e-08,9.974841e-07,5.486162e-08,2.992452e-09,4.087690e-08,4.155519e-08,7.556141e-07,3.989936e-09,3.989936e-09,1.396478e-08,9.974841e-10,9.974841e-10,0.0,3.836477e-10,9.974841e-09,3.989936e-08,2.992452e-09,0.0,2.892704e-08,0.000005,0.000015,0.000019,0.002432,0.000018,6.891373e-09,5.486162e-11,1.828721e-08,0.000000e+00


Now we are ready to split the data set to test and train and implement a model!

Resources: 
* https://www.kaggle.com/shivamb/an-insightful-story-of-crowdfunding-projects
* https://www.kaggle.com/srishti280992/kickstarter-project-classification-lgbm-70-3
* https://www.kaggle.com/dronqo/how-successful-is-your-kickstarter-project
* https://www.kaggle.com/kromel/kickstarter-successful-vs-failed
* https://www.kaggle.com/carlolepelaars/exploration-of-kickstarter-data-2010-2017